In [1]:
import numpy as np
import math 
import pandas as pd
import re

### Lagerdaten (Festlager)

In [2]:
#Dynamische Tragzahl [N]
Cr=32500
#Statische Tragzahl [N]
Cor=19000
#Berechnungsfaktor
fo=13.8

#Hauptabmessungen

#Breite[mm]
B=13

#Bohrungsdurchmesser[mm]
d=30

#Außendurchmesser[mm]
D=55

# Drehzahl [min^-1]
n=350

### Dynamische Äquivalente Lagerbelastung P 

In [3]:
#Resultierende Radiallast[N]

FAy=330.80
FAz=3661.20

FR = math.sqrt((FAy)**2 + (FAz)**2)
print(f'Die resultierende Radiallast ist: {FR:4.4f} N')

#Axiallast[N]
FA=0
print("Die resultierende Axiallast ist:",FA,"N")

Die resultierende Radiallast ist: 3676.1140 N
Die resultierende Axiallast ist: 0 N


### Berechnungsfaktoren X, Y für Rillenkugellager für den Faktor CN

In [4]:
d = {'Q': [0.3, 0.5, 0.9, 1.6, 3, 6], 'e': [0.22, 0.24, 0.28, 0.32, 0.36, 0.43], '[X] fo*FA/Cor <= e': [1, 1, 1, 1, 1,1],
                  '[Y] fo*FA/Cor <= e': [0, 0, 0, 0, 0,0], '[X] fo*FA/Cor > e': [0.56, 0.56, 0.56, 0.56, 0.56,0.56], '[Y] fo*FA/Cor > e': [2, 1.8, 1.58, 1.4, 1.2,1] }

df = pd.DataFrame(data=d)
# df.set_index('fo*FA/Cor', inplace = True)

display(df)

,Q,e,[X] fo*FA/Cor <= e,[Y] fo*FA/Cor <= e,[X] fo*FA/Cor > e,[Y] fo*FA/Cor > e
0,0.3,0.22,1,0,0.56,2.00
1,0.5,0.24,1,0,0.56,1.80
2,0.9,0.28,1,0,0.56,1.58
3,1.6,0.32,1,0,0.56,1.40
4,3.0,0.36,1,0,0.56,1.20
5,6.0,0.43,1,0,0.56,1.00


In [7]:
#Berechnung vom Faktor FA/FR
faktor_FA_FR= FA/FR
print(f'Der Faktor FA/FR ist {faktor_FA_FR:4.4f}')

#Berechnung des Quotienten [Q]
Q = (fo*FA)/Cor

#Interpolation und Auswahl von e(Q)

faktoren_Q = df['Q'].to_numpy()
faktoren_e = df['e'].to_numpy()
faktoren_y = df['[Y] fo*FA/Cor > e'].to_numpy()
# print(faktoren_Q)

for i in range(len(faktoren_Q)):
    if faktoren_Q[i] == Q:
        e=Q
        y=faktoren_y[i]
        break
    elif faktoren_Q[i]< Q and Q < faktoren_Q[i+1]:
        e=((faktoren_e[i+1]-faktoren_e[i])/(faktoren_Q[i+1]-faktoren_Q[i]))*(Q-faktoren_Q[i])+faktoren_e[i]
        y=((faktoren_y[i+1]-faktoren_y[i])/(faktoren_Q[i+1]-faktoren_Q[i]))*(Q-faktoren_Q[i])+faktoren_y[i]
        
    print(f'Der Wert von e (interpoliert) ist: {e:4.4f}')
    print(f'Der Wert von y (interpoliert) ist: {y:4.4f}')

#Fallunterscheidung für die Berechnung von P

x= 0.56 #Konstant
if faktor_FA_FR > e:
    #Überprüfung mit den Nachkommastellen
    y=float('%.3f' % (y))
    FR=float('%.3f' % (FR))
    FA=float('%.3f' % (FA))
    
    #Lagerbelastung
    P= x*FR + y*FA
    print(f'Der Wert der äquivalenten Lagerbelastung ist: {P:4.4f} N')
else:
    #Lagerbelastung
    P=FR
    print(f'Der Wert der äquivalenten Lagerbelastung ist: {P:4.4f} N')

Der Faktor FA/FR ist 0.0000


NameError: name 'e' is not defined

### Nominelle Lagerlebensdauer in Stunden

In [ ]:
# P=float('%.3f' % (P))
L10h= (10**6/(60*n))*((Cr/P)**3)
print(f'Der nominelle Lagerlebensdauer in Stunden ist:{L10h:4.4f}h')

### Nominelle Lagerlebensdauer in Umdrehungen

In [ ]:
L10=(Cr/P)**3
print(f'Der nominelle Lagerlebensdauer in Umdrehungen ist: {L10:4.4f} * 10^6 Umdrehungen')

### Erweiterte Lagerlebensdauer

In [ ]:
a1=1
a_iso=1
Lna_u=a1*a_iso*L10
Lna_h=a1*a_iso*L10h
print(f'Die erweiterte modfizierte Lagerlebensdauer in Umdrehungen ist: {Lna_u:4.4f} * 10^6 Umdrehungen')
print(f'Der nominelle Lagerlebensdauer in Stunden ist:{Lna_h:4.4f} h')